In [87]:
%load_ext autoreload
%autoreload

import os
import sys
import random
import django
from IPython.core.debugger import set_trace
from pathlib import Path
from django.core.files.base import ContentFile
from rest_framework import serializers
from datetime import datetime
import time
from collections import Counter
import pandas as pd
import random
from tqdm import tqdm_notebook

parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)
django.setup()

import getch.models as m
from django.apps import apps
from django_currentuser.middleware import get_current_user
from django.db.models import F, Q, Sum, Count, Case, When, IntegerField, Value

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
%%time
tags = m.Tag.objects.all()
random.sample(list(tags.values_list('id', flat=True)), 2)

CPU times: user 4.09 ms, sys: 1.14 ms, total: 5.23 ms
Wall time: 206 ms


[1195, 2068]

In [58]:
%%time
m.Tag.objects.order_by('?')[:2].values_list('id', flat=True)

CPU times: user 121 µs, sys: 0 ns, total: 121 µs
Wall time: 124 µs


<QuerySet [8103, 4176]>

In [54]:
%%time
tags = m.Tag.objects.all()
samples = random.sample(range(0, tags.count()), 2)
tags[samples[0]], tags[samples[1]]

CPU times: user 2.73 ms, sys: 806 µs, total: 3.53 ms
Wall time: 525 ms


(<Tag: [167] clothing | 팬츠 | 와이드팬츠>, <Tag: [4231] clothing | 스커트 | 스커트>)

In [89]:
%%time
bgr = m.BalancegameRecord.objects.exclude(tag_pos__isnull=True).select_related('tag_pos', 'tag_neg')

df_pos = pd.DataFrame([r.tag_pos.__dict__ for r in bgr])
df_neg = pd.DataFrame([r.tag_neg.__dict__ for r in bgr])

flds = [
    'type','category','item',
    'color','detail','pattern','texture','look','length','sleeve_length',
    'neckline','fit','shape','heel_height','heel_shape','toe_type','sole_type',
    'strap','size','main_material','sub_material'
]

count = {col:Counter(df_pos[col])-Counter(df_neg[col]) for col in flds }; count

CPU times: user 13.9 ms, sys: 1.3 ms, total: 15.2 ms
Wall time: 312 ms


{'type': Counter(),
 'category': Counter(),
 'item': Counter({'무스탕': 1, '뮬': 1, '야상': 1, '캐미솔': 1, '캡': 1}),
 'color': Counter({'스카이블루': 2,
          '화이트': 6,
          '카키': 2,
          '베이지': 1,
          '오렌지': 1,
          '퍼플': 1,
          '브라운': 3,
          '실버': 1}),
 'detail': Counter({'드롭숄더': 7,
          '컷아웃': 1,
          '포켓': 10,
          '시스루': 1,
          '스트랩': 2,
          '후드': 2,
          '집업': 1,
          '자수': 1,
          '체인': 1}),
 'pattern': Counter({'스트라이프': 1, '레터링': 2, '그라데이션': 1, '체크': 1, '메탈릭': 1}),
 'texture': Counter({'면': 4, '데님': 1, '가죽': 1, None: 1, '메탈': 1}),
 'look': Counter({'페미닌': 4}),
 'length': Counter({'롱': 1, '하프': 2}),
 'sleeve_length': Counter({'반팔': 4}),
 'neckline': Counter({'차이나카라': 2, '스퀘어넥': 1, '브이넥': 1, '노카라': 1}),
 'fit': Counter({'오버사이즈': 18}),
 'shape': Counter({'테이퍼드': 1, '사각형': 1}),
 'heel_height': Counter(),
 'heel_shape': Counter(),
 'toe_type': Counter({'아몬드토': 1}),
 'sole_type': Counter(),
 'strap': Counter(),
 'size'

In [91]:
sum(count.values(), Counter())

Counter({'무스탕': 1,
         '뮬': 1,
         '야상': 1,
         '캐미솔': 1,
         '캡': 1,
         '스카이블루': 2,
         '화이트': 6,
         '카키': 2,
         '베이지': 1,
         '오렌지': 1,
         '퍼플': 1,
         '브라운': 3,
         '실버': 3,
         '드롭숄더': 7,
         '컷아웃': 1,
         '포켓': 10,
         '시스루': 1,
         '스트랩': 2,
         '후드': 2,
         '집업': 1,
         '자수': 1,
         '체인': 1,
         '스트라이프': 1,
         '레터링': 2,
         '그라데이션': 1,
         '체크': 1,
         '메탈릭': 1,
         '면': 4,
         '데님': 1,
         '가죽': 2,
         None: 1,
         '메탈': 1,
         '페미닌': 4,
         '롱': 1,
         '하프': 2,
         '반팔': 4,
         '차이나카라': 2,
         '스퀘어넥': 1,
         '브이넥': 1,
         '노카라': 1,
         '오버사이즈': 18,
         '테이퍼드': 1,
         '사각형': 1,
         '아몬드토': 1})

In [80]:
df_pos[['type','category','item','detail','fit']]

,type,category,item,detail,fit
0,clothing,캐주얼상의,티셔츠,드롭숄더,오버사이즈
1,clothing,캐주얼상의,티셔츠,드롭숄더,노멀
2,clothing,캐주얼상의,티셔츠,컷아웃,노멀
3,clothing,캐주얼상의,티셔츠,드롭숄더,노멀
4,clothing,캐주얼상의,티셔츠,드롭숄더,노멀
5,clothing,캐주얼상의,티셔츠,드롭숄더,오버사이즈
6,clothing,캐주얼상의,티셔츠,드롭숄더,오버사이즈
7,clothing,캐주얼상의,티셔츠,드롭숄더,오버사이즈
8,clothing,캐주얼상의,티셔츠,None,오버사이즈
9,clothing,캐주얼상의,티셔츠,포켓,노멀


In [34]:
# type = models.CharField(max_length=20, null=False, blank=False)
# category = models.CharField(max_length=20, null=False, blank=False)
# item = models.CharField(max_length=20, null=False, blank=False)
# color = models.CharField(max_length=20, null=True, blank=True)
# detail = models.CharField(max_length=20, null=True, blank=True)
# pattern = models.CharField(max_length=20, null=True, blank=True) # 옴니어스 원래의 속성은 print임
# texture = models.CharField(max_length=20, null=True, blank=True)
# look = models.CharField(max_length=20, null=True, blank=True)
# length = models.CharField(max_length=20, null=True, blank=True)
# sleeve_length = models.CharField(max_length=20, null=True, blank=True)
# neckline = models.CharField(max_length=20, null=True, blank=True)
# fit = models.CharField(max_length=20, null=True, blank=True)
# shape = models.CharField(max_length=20, null=True, blank=True)
# heel_height = models.CharField(max_length=20, null=True, blank=True)
# heel_shape = models.CharField(max_length=20, null=True, blank=True)
# toe_type = models.CharField(max_length=20, null=True, blank=True)
# sole_type = models.CharField(max_length=20, null=True, blank=True)

# strap = models.BooleanField(default=False)
# size = models.CharField(max_length=20, null=True, blank=True)
# main_material = models.CharField(max_length=20, null=True, blank=True)
# sub_material = models.CharField(max_length=20, null=True, blank=True)

In [97]:
univ = [
    [10, {'type':'clothing', 'category':'캐주얼상의'}],
    [10, {'type':'clothing', 'category':'팬츠'}],
    [10, {'type':'clothing', 'category':'재킷'}],
    [10, {'type':'clothing', 'category':'니트웨어'}],
    [10, {'type':'clothing', 'category':'청바지'}],
    [10, {'type':'clothing', 'category':'스커트'}],
    [10, {'type':'clothing', 'category':'셔츠'}],
    [10, {'type':'clothing', 'category':'점퍼'}],
    [10, {'type':'clothing', 'category':'코트'}],
    [10, {'type':'clothing', 'category':'드레스'}],
    [10, {'type':'clothing', 'category':'블라우스'}],
    [10, {'type':'clothing', 'category':'탑'}],
    [10, {'type':'clothing', 'category':'패딩'}],
    [10, {'type':'clothing', 'category':'베스트'}],
    [10, {'type':'clothing', 'category':'점프수트'}],
    [10, {'type':'clothing', 'category':'수영복'}],
    
    [7, {'type':'shoes'}],
    [7, {'type':'shoes', 'category':'운동화'}],
    [7, {'type':'shoes', 'category':'부츠/워커'}],
    [7, {'type':'shoes', 'category':'로퍼'}],
    [7, {'type':'shoes', 'category':'슬리퍼'}],
    [7, {'type':'shoes', 'category':'펌프스'}],
    [7, {'type':'shoes', 'category':'정장구두'}],
    [7, {'type':'shoes', 'category':'샌들'}],
    
    [5, {'type':'bags', 'category':'가방'}],
    [1, {'type':'bags', 'category':'지갑'}],
    
    [1, {'type':'accessories', 'category':'잡화', 'item':'선글라스'}],
    [1, {'type':'accessories', 'category':'잡화', 'item':'벨트'}],
    [1, {'type':'accessories', 'category':'잡화', 'item':'안경'}],
    [1, {'type':'accessories', 'category':'잡화', 'item':'손목시계'}],
    [1, {'type':'accessories', 'category':'잡화', 'item':'머플러'}],
    [1, {'type':'accessories', 'category':'잡화', 'item':'헤어밴드'}],
    [1, {'type':'accessories', 'category':'잡화', 'item':'스카프'}],
    [1, {'type':'accessories', 'category':'잡화', 'item':'양말'}],
    [1, {'type':'accessories', 'category':'잡화', 'item':'헤어악세서리'}],
#     {'type':'accessories', 'category':'잡화', 'item':'고글'},
#     {'type':'accessories', 'category':'잡화', 'item':'넥워머'},
#     {'type':'accessories', 'category':'잡화', 'item':'넥타이'},
#     {'type':'accessories', 'category':'잡화', 'item':'타이츠스타킹'},
#     {'type':'accessories', 'category':'잡화', 'item':'암워머'},
    
    [1, {'type':'jewelries'}],
    [1, {'type':'jewelries', 'category':'목걸이'}],
    [1, {'type':'jewelries', 'category':'팔찌/발찌'}],
    [1, {'type':'jewelries', 'category':'귀걸이/피어싱'}],
    [1, {'type':'jewelries', 'category':'반지'}],
#     {'type':'jewelries', 'category':'펜던트'},
    
    [3, {'type':'hats', 'category':'모자'}],
]


In [98]:
df_choice = pd.DataFrame(univ); df_choice

,0,1
0,10,"{'type': 'clothing', 'category': '캐주얼상의'}"
1,10,"{'type': 'clothing', 'category': '팬츠'}"
2,10,"{'type': 'clothing', 'category': '재킷'}"
3,10,"{'type': 'clothing', 'category': '니트웨어'}"
4,10,"{'type': 'clothing', 'category': '청바지'}"
5,10,"{'type': 'clothing', 'category': '스커트'}"
6,10,"{'type': 'clothing', 'category': '셔츠'}"
7,10,"{'type': 'clothing', 'category': '점퍼'}"
8,10,"{'type': 'clothing', 'category': '코트'}"
9,10,"{'type': 'clothing', 'category': '드레스'}"


In [103]:
random.choices(df_choice[1], weights=df_choice[0])

[{'type': 'clothing', 'category': '블라우스'}]

In [105]:
m.Tag.objects.filter(**random.choices(df_choice[1], weights=df_choice[0])[0])

<QuerySet [<Tag: [4412] clothing | 팬츠 | 스웨트팬츠>, <Tag: [4411] clothing | 팬츠 | 슬랙스>, <Tag: [4410] clothing | 팬츠 | 스웨트팬츠>, <Tag: [4404] clothing | 팬츠 | 슬랙스>, <Tag: [4401] clothing | 팬츠 | 와이드팬츠>, <Tag: [4400] clothing | 팬츠 | 와이드팬츠>, <Tag: [4398] clothing | 팬츠 | 슬랙스>, <Tag: [4387] clothing | 팬츠 | 슬랙스>, <Tag: [4376] clothing | 팬츠 | 카고팬츠>, <Tag: [4375] clothing | 팬츠 | 치노팬츠>, <Tag: [4374] clothing | 팬츠 | 슬랙스>, <Tag: [4358] clothing | 팬츠 | 스웨트팬츠>, <Tag: [4353] clothing | 팬츠 | 슬랙스>, <Tag: [4351] clothing | 팬츠 | 와이드팬츠>, <Tag: [4350] clothing | 팬츠 | 치노팬츠>, <Tag: [4349] clothing | 팬츠 | 스웨트팬츠>, <Tag: [4347] clothing | 팬츠 | 슬랙스>, <Tag: [4345] clothing | 팬츠 | 슬랙스>, <Tag: [4341] clothing | 팬츠 | 와이드팬츠>, <Tag: [4341] clothing | 팬츠 | 슬랙스>, '...(remaining elements truncated)...']>